In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
df = pd.read_csv('EHB_ACTIVE_GRANT_FA_AGR.csv')

In [5]:
# df.head()
# df.columns

In [4]:
# only keep relevant columns
grant_data = df[['Award Year', 'Total Active Grant Financial Assistance', 'Grantee City', 'State Name', 'Grantee ZIP Code', 'Uniform Data System Grant Program Description', 'Grantee County Name', 'Complete County Name','HRSA Region']]


In [5]:
grant_data.tail()

,Award Year,Total Active Grant Financial Assistance,Grantee City,State Name,Grantee ZIP Code,Uniform Data System Grant Program Description,Grantee County Name,Complete County Name,HRSA Region
24213,2021,148587.0,Astoria,Oregon,97103-3329,This notice announces the opportunity to apply...,Clatsop,Clatsop County,Region 10
24214,2021,49529.0,Jonesboro,Arkansas,72404-6897,This notice announces the opportunity to apply...,Craighead,Craighead County,Region 6
24215,2020,300000.0,Bamberg,South Carolina,29003-8361,NaN,Bamberg,Bamberg County,Region 4
24216,2021,300000.0,Bamberg,South Carolina,29003-8361,NaN,Bamberg,Bamberg County,Region 4
24217,2021,49529.0,Woodville,Texas,75979-4738,This notice announces the opportunity to apply...,Tyler,Tyler County,Region 6


In [9]:
#filter for only awards in 2019 
grant_data_2015_2019 = grant_data[(grant_data['Award Year'] >= 2015) & (grant_data['Award Year'] <= 2019)]
grant_data_2015_2019 = grant_data_2015_2019.reset_index()

#dropna values 
grant_data_2015_2019 = grant_data_2015_2019.dropna()

In [ ]:
# now we want to get county level data
# go from county to population size and determine funding for large vs small counties 

In [15]:
cbsa = pd.read_csv('cbsa2fipsxw.csv')
cbsa.head()

,cbsacode,metrodivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33860.0,NaN,NaN,"Montgomery, AL",Metropolitan Statistical Area,NaN,NaN,Autauga County,Alabama,1.0,1.0,Central
2,19300.0,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1.0,3.0,Central
3,13820.0,NaN,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,NaN,"Birmingham-Hoover-Talladega, AL",Bibb County,Alabama,1.0,7.0,Outlying
4,13820.0,NaN,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,NaN,"Birmingham-Hoover-Talladega, AL",Blount County,Alabama,1.0,9.0,Outlying


In [23]:
grant_data_2015_2019.head()

,index,Award Year,Total Active Grant Financial Assistance,Grantee City,State Name,Grantee ZIP Code,Uniform Data System Grant Program Description,Grantee County Name,Complete County Name,HRSA Region,Mental Health Assistance
0,1,2019,225910.8,Milwaukee,Wisconsin,53226-3548,Loans for HPSL graduates,Milwaukee,Milwaukee County,Region 5,0.0
2,11,2017,237959.0,Springfield,Illinois,62702-4933,Primary Care Loans (PCL) to fund Allopathic Me...,Sangamon,Sangamon County,Region 5,0.0
3,12,2019,250000.0,Springfield,Illinois,62702-4933,Primary Care Loans (PCL) to fund Allopathic Me...,Sangamon,Sangamon County,Region 5,0.0
4,14,2018,5562.0,Saint Louis,Missouri,63147-2912,This notice announces the opportunity for reci...,St. Louis City,St. Louis City,Region 7,0.0
5,15,2019,750000.0,Saint Louis,Missouri,63147-2912,This notice announces the opportunity for reci...,St. Louis City,St. Louis City,Region 7,0.0


In [17]:
# get grant assistance per state 
# grants_per_county = grant_data_2019.groupby(by='Complete County Name')['Total Active Grant Financial Assistance'].sum().reset_index()
# grants_per_county.head()

grant_data_2015_2019['Mental Health Assistance'] = grant_data_2015_2019.apply(lambda row: row['Total Active Grant Financial Assistance']\
                                                                               if 'mental health' in row['Uniform Data System Grant Program Description'] else 0, axis =1)

grants_per_county = grant_data_2015_2019.groupby(by=['Complete County Name', 'State Name'])[['Total Active Grant Financial Assistance', 'Mental Health Assistance']].sum().reset_index()
grants_per_county.head()



,Complete County Name,State Name,Total Active Grant Financial Assistance,Mental Health Assistance
0,Accomack County,Virginia,10423990.0,0.0
1,Ada County,Idaho,10014823.0,720000.0
2,Adair County,Missouri,3364332.0,0.0
3,Adams County,Colorado,1077388.0,260000.0
4,Adams County,Idaho,2834558.0,0.0


In [18]:
cbsa_data = cbsa[['countycountyequivalent', 'centraloutlyingcounty', 'statename', 'cbsacode']]

#cbsa_data.head()
# merge total grants and mental health grants
grants_per_county_cbsa = grants_per_county.merge(cbsa_data, how='left', left_on='Complete County Name', right_on='countycountyequivalent')

grants_per_county_cbsa.head()

,Complete County Name,State Name,Total Active Grant Financial Assistance,Mental Health Assistance,countycountyequivalent,centraloutlyingcounty,statename,cbsacode
0,Accomack County,Virginia,10423990.0,0.0,NaN,NaN,NaN,NaN
1,Ada County,Idaho,10014823.0,720000.0,Ada County,Central,Idaho,14260.0
2,Adair County,Missouri,3364332.0,0.0,Adair County,Central,Missouri,28860.0
3,Adams County,Colorado,1077388.0,260000.0,Adams County,Central,Colorado,19740.0
4,Adams County,Colorado,1077388.0,260000.0,Adams County,Central,Illinois,39500.0


# # @Steph, the grants_per_county_cbsa is the df we want to visualize

In [19]:
grants_per_county_cbsa = grants_per_county.merge(cbsa_data, how='inner', left_on=['Complete County Name', 'State Name'], right_on=['countycountyequivalent', 'statename'])


grants_per_county_cbsa['Outlying'] = grants_per_county_cbsa['centraloutlyingcounty'].apply(lambda x: 1 if x =='Outlying' else 0)

grants_per_county_cbsa.tail(10)

,Complete County Name,State Name,Total Active Grant Financial Assistance,Mental Health Assistance,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,Outlying
702,Yamhill County,Oregon,1323089.0,0.0,Yamhill County,Outlying,Oregon,38900.0,1
703,Yates County,New York,381447.0,0.0,Yates County,Outlying,New York,40380.0,1
704,Yavapai County,Arizona,4230713.0,0.0,Yavapai County,Central,Arizona,39140.0,0
705,Yellowstone County,Montana,9541555.0,0.0,Yellowstone County,Central,Montana,13740.0,0
706,Yolo County,California,16361489.0,699965.0,Yolo County,Outlying,California,40900.0,1
707,York County,Maine,2541013.0,0.0,York County,Central,Maine,38860.0,0
708,York County,Pennsylvania,4298419.0,0.0,York County,Central,Pennsylvania,49620.0,0
709,York County,South Carolina,4883212.0,0.0,York County,Outlying,South Carolina,16740.0,1
710,Yuba County,California,2443802.0,0.0,Yuba County,Central,California,49700.0,0
711,Yuma County,Arizona,6683502.0,0.0,Yuma County,Central,Arizona,49740.0,0


In [20]:
grants_per_county_cbsa[["Outlying", "Total Active Grant Financial Assistance", 'Mental Health Assistance']].corr()

,Outlying,Total Active Grant Financial Assistance,Mental Health Assistance
Outlying,1.000000,-0.085741,-0.058443
Total Active Grant Financial Assistance,-0.085741,1.000000,0.401111
Mental Health Assistance,-0.058443,0.401111,1.000000


In [21]:
outlying = grants_per_county_cbsa[grants_per_county_cbsa['Outlying'] == 1]

central = grants_per_county_cbsa[grants_per_county_cbsa['Outlying'] == 0]

outlying[['Total Active Grant Financial Assistance','Mental Health Assistance']].describe()

,Total Active Grant Financial Assistance,Mental Health Assistance
count,8.200000e+01,82.000000
mean,3.557289e+06,31673.853659
std,4.233144e+06,139724.225906
min,3.881000e+03,0.000000
25%,6.814798e+05,0.000000
50%,2.205296e+06,0.000000
75%,5.062036e+06,0.000000
max,2.126549e+07,725000.000000


In [22]:
central[['Total Active Grant Financial Assistance','Mental Health Assistance']].describe()

,Total Active Grant Financial Assistance,Mental Health Assistance
count,6.300000e+02,6.300000e+02
mean,1.321745e+07,7.866136e+04
std,3.809418e+07,2.679446e+05
min,4.360000e+03,0.000000e+00
25%,1.211502e+06,0.000000e+00
50%,4.128082e+06,0.000000e+00
75%,9.924230e+06,0.000000e+00
max,4.439880e+08,3.620438e+06
